# JAX-Fluids: Case setup demo
This demo will show what a typical case setup file contains and how you can change the case setup for your simulation. We use the case setup "01_case_setup_sod.json" from an earlier tutorial.

In [ ]:
import json
from jaxfluids import InputReader, Initializer, SimulationManager
from jaxfluids.post_process import load_data, create_lineplot

## Case Setup
In JAX-Fluids we typically specify the case setup via a json file. In the first tutorial we have seen that we can pass the path to any case setup as a string to the InputReader. Internally, the InputReader will read the json file and convert it to a dictionary.

In [ ]:
input_reader = InputReader("01_case_setup_sod.json", "01_numerical_setup_sod.json")

However, it is also possible to pass a dictionary directly to the InputReader. Let us load the 01_case_setup_sod.json file and look what it contains.

In [ ]:
case_setup = json.load(open("01_case_setup_sod.json"))
case_setup.keys()

The case setup specifies general simulation properties, domain properties, boundary conditions, initial conditions, material properties and parameters for non dimensionalization. Let us have a look at the domain properties.

In [ ]:
case_setup["domain"]

The domain properties specify the physical extent of the simulation domain and the spatial resolution. Currently, we use 100 points to discretize the domain. Let's change that value to 200 and run a simulation.

In [ ]:
case_setup["domain"]["x"]["cells"] = 200
case_setup["domain"]

## Simulation Setup
To run a forward simulation with JAX-Fluids, you need an InputRead, an Initializer, and a SimulationManager.

The InputReader reads a case setup file and a numerical setup file. Now, we pass the updated case setup dictionary to the InputReader.

In [ ]:
input_reader = InputReader(case_setup, "01_numerical_setup_sod.json")
initializer  = Initializer(input_reader)
sim_manager  = SimulationManager(input_reader)

## Run Simulation
The Initializer creates a dictionary which contains all initial flow fields. The SimulationManager runs the simulation and writes h5 output files.

In [ ]:
buffer_dictionary = initializer.initialization()
sim_manager.simulate(buffer_dictionary)

As you can see, the spatial resolution has been changed to 200 points in x-direction. Let us load the data and visualize the results.

## Load Data
JAX-Fluids offers post-processing tools for visualizing your simulation data. You can find them under jaxfluids.post_process.

In [ ]:
path = sim_manager.output_writer.save_path_domain
quantities = ["density", "velocityX", "pressure"]
cell_centers, cell_sizes, times, data_dict = load_data(path, quantities)

## Visualize Data

In [ ]:
nrows_ncols = (1,3)
create_lineplot(data_dict, cell_centers, times, nrows_ncols=nrows_ncols, interval=100, static_time=0.2)